<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/CargaGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Celda 1: Instalación de Librerías
!pip install earthengine-api geemap --quiet

import ee
import geemap
from IPython.display import display, Markdown

# La autenticación ya se realizó. Solo importamos ee para usarlo.
import ee
print("✅ Librerías y paquetes importados.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00
✅ Librerías y paquetes importados.


In [2]:

# Celda Final: Autenticación, Inicialización y Prueba de Uso

import ee
from IPython.display import display, Markdown

# ⚠️ Usamos tu ID de proyecto.
PROYECTO_ID = "gen-lang-client-0528470346"

# 1. FORZAR LA AUTENTICACIÓN (pedirá un nuevo enlace y código)
display(Markdown("### ⚠️ Paso 1: Forzar Autenticación"))
display(Markdown("Si no aparece el enlace, reinicia e inténtalo de nuevo."))

# El comando 'ee.Authenticate' debería mostrar el enlace y pedir el código.
ee.Authenticate()

# 2. Inicialización
try:
    display(Markdown("### ✅ Paso 2: Inicializando con Proyecto..."))
    ee.Initialize(project=PROYECTO_ID)

    # Prueba rápida para confirmar que GEE está listo
    test_geometry = ee.Geometry.Point(-71.146, 8.587)
    display(Markdown("✅ **¡ÉXITO! Google Earth Engine está listo para usar.**"))

except Exception as e:
    display(Markdown(f"❌ **Error Fatal:** La inicialización falló. El problema debe ser un permiso faltante en el proyecto **`{PROYECTO_ID}`** dentro de la consola de Google Cloud, a pesar de que la API esté habilitada."))

### ⚠️ Paso 1: Forzar Autenticación

Si no aparece el enlace, reinicia e inténtalo de nuevo.

### ✅ Paso 2: Inicializando con Proyecto...

✅ **¡ÉXITO! Google Earth Engine está listo para usar.**

In [3]:

# Celda Cero: Inicialización de GEE
# Sustituye con tu ID de proyecto final
import ee
PROYECTO_ID = "gen-lang-client-0528470346"

try:
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Earth Engine inicializado. Listo para el análisis.")
except Exception as e:
    print(f"❌ Error de inicialización final. Asegúrate de reiniciar el entorno de ejecución. Error: {e}")

✅ Earth Engine inicializado. Listo para el análisis.


In [3]:

# Celda 2: Obtención y Procesamiento de la Imagen Sentinel-2

# 1. Definir la ubicación de interés (Mérida, Venezuela)
lon = -71.146
lat = 8.587
# Esto ahora funciona porque GEE está inicializado
punto_merida = ee.Geometry.Point(lon, lat)

# 2. Definir el rango de tiempo y Colección Sentinel-2
fecha_inicio = '2024-01-01'
fecha_fin = '2024-03-01'

# Filtramos la colección por fecha, área y porcentaje de nubes bajo (<10%)
coleccion_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(fecha_inicio, fecha_fin) \
    .filterBounds(punto_merida) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# 3. Crear el mosaico compuesto (media de píxeles menos nublados) y recortarlo a 30 km
imagen_compuesta = coleccion_s2.median().clip(punto_merida.buffer(30000))

print("✅ Datos Sentinel-2 procesados (mosaico compuesto listo).")

✅ Datos Sentinel-2 procesados (mosaico compuesto listo).


In [7]:

# Celda 3 Revisada: Visualización con Centrado Forzado

from IPython.display import display, Markdown
import geemap

# Parámetros de la ubicación de Mérida (tomados de Celda 2)
lon = -71.146
lat = 8.587
zoom_level = 10 # Nivel de zoom deseado

# 1. Parámetros de Visualización (Color Natural)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# 2. Crear el objeto Mapa (location y zoom son ignorados a veces)
m = geemap.Map()

# 3. FORZAR el centrado del mapa usando el método setCenter
# Este método es más robusto que usar el constructor.
m.setCenter(lon, lat, zoom_level)

# 4. Añadir la capa de imagen satelital de GEE
# Asegúrate de que 'imagen_compuesta' fue definida en la Celda 2
m.addLayer(
    ee_object=imagen_compuesta,
    vis_params=vis_params,
    name='Sentinel-2 Compuesta (Color Natural)'
)

# 5. Mostrar el mapa
m.add_layer_control()
display(Markdown("### Resultado: Mapa de Mérida Centrado Forzado"))
m # Muestra el mapa interactivo

### Resultado: Mapa de Mérida Centrado Forzado

Map(center=[8.587, -71.146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…